In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
from datetime import datetime

In [19]:
df = pd.read_csv('sales_data_all.csv')

In [20]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df["quarter"] = df.sale_date.dt.quarter
df["profit"] = (df['regular_price'] - df['purchase_price']) * df['quantity']
df

,sale_date,product_id,quantity,sales_price_tax,regular_price_tax,sales_price,regular_price,purchase_price,quarter,profit
0,2021-01-02,109751,1,2499.0,2777.0,2082.0,2313.8917,1499.4000,1,814.4917
1,2021-01-02,110853,1,3990.0,8322.0,3325.0,6935.1852,2833.7000,1,4101.4852
2,2021-01-02,131410,1,2288.0,3767.0,1907.0,3138.8889,1860.0000,1,1278.8889
3,2021-01-02,134294,1,4990.0,6666.0,4158.0,5554.6296,3599.4024,1,1955.2272
4,2021-01-02,134298,1,5990.0,7777.0,4992.0,6480.5583,4199.4000,1,2281.1583
...,...,...,...,...,...,...,...,...,...,...
119120,2021-12-31,184595,2,3290.0,4433.0,2742.0,3694.4444,2474.2500,4,2440.3888
119121,2021-12-31,184647,1,1990.0,2878.0,1658.0,2398.1481,1638.0000,4,760.1481
119122,2021-12-31,184749,1,4990.0,5544.0,4158.0,4620.3704,3600.0000,4,1020.3704
119123,2021-12-31,184809,1,2590.0,2878.0,2158.0,2398.1481,1650.0000,4,748.1481


In [21]:
df = df.groupby(['quarter','product_id'], as_index=False)['quantity'].sum()
df

,quarter,product_id,quantity
0,1,100043,3
1,1,100044,2
2,1,100108,1
3,1,100122,4
4,1,100124,8
...,...,...,...
21014,4,184647,7
21015,4,184749,5
21016,4,184809,13
21017,4,184860,21


In [22]:
df['pop_index'] = df['quantity'] / df['quantity'].sum()
df['pop_index_perc'] = (df['pop_index'] - df['pop_index'].min())\
                                            / (df['pop_index'].max() - df['pop_index'].min())
df

,quarter,product_id,quantity,pop_index,pop_index_perc
0,1,100043,3,0.000018,0.003992
1,1,100044,2,0.000012,0.001996
2,1,100108,1,0.000006,0.000000
3,1,100122,4,0.000024,0.005988
4,1,100124,8,0.000048,0.013972
...,...,...,...,...,...
21014,4,184647,7,0.000042,0.011976
21015,4,184749,5,0.000030,0.007984
21016,4,184809,13,0.000079,0.023952
21017,4,184860,21,0.000127,0.039920


In [50]:
df_sd = df.groupby('product_id')['pop_index_perc'].std(ddof=0).to_frame()
df_sd

,pop_index_perc
product_id,
100043,0.001996
100044,0.001655
100052,0.000000
100070,0.002489
100078,0.004705
...,...
184647,0.000000
184749,0.000000
184809,0.000000


In [51]:
df_rank = df_sd.sort_values('pop_index_perc', ascending = False).head(10)
df_rank.reset_index(inplace = True)
df_rank

,product_id,pop_index_perc
0,110853,0.407522
1,112582,0.294303
2,110675,0.228172
3,156095,0.227761
4,126707,0.206285
5,160649,0.181218
6,156485,0.174946
7,136250,0.158964
8,157913,0.155413
9,142254,0.153020


In [35]:
prod_rank = list(df_rank['product_id'])
prod_rank

[110853,
 112582,
 110675,
 156095,
 126707,
 160649,
 156485,
 136250,
 157913,
 142254]

In [42]:
df_top = df[df['product_id'].isin(prod_rank)]
df_top

,quarter,product_id,quantity,pop_index,pop_index_perc
637,1,110675,3,0.000018,0.003992
649,1,110853,502,0.003036,1.000000
1068,1,126707,4,0.000024,0.005988
1882,1,136250,26,0.000157,0.049900
2339,1,142254,34,0.000206,0.065868
3759,1,156095,258,0.001560,0.512974
3860,1,156485,9,0.000054,0.015968
4201,1,157913,28,0.000169,0.053892
5412,2,110675,19,0.000115,0.035928
5425,2,110853,138,0.000835,0.273453


In [71]:
fig = px.line(df_top, x="quarter", y="pop_index_perc", color='product_id', title='Top 10 products with highe standard deviation in popularity index between quarters')
fig